In [7]:
import dask.dataframe as dd
from pathlib import Path
import dask
from dask.distributed import Client

In [5]:
# --- Конфигурация путей ---
DATA_DIR_PUBCH = Path("../data/pubchem_compounds.parquet.dask")
DATA_DIR_DRUGS = Path("../data/drugspacex.parquet.dask")


print("config:")
print(f"  pubchem Parquet: \t{DATA_DIR_PUBCH.name}")
print(f"  drugspacex Parquet: \t{DATA_DIR_DRUGS.name}")

config:
  pubchem Parquet: 	pubchem_compounds.parquet.dask
  drugspacex Parquet: 	drugspacex.parquet.dask


In [8]:
# --- Шаг 0: Настройка Dask (Рекомендуется) ---
# Это запустит локальный кластер на вашей машине, что даст прирост производительности
# и хороший дашборд для мониторинга по адресу http://127.0.0.1:8787/status
client = Client()
print(f"Dask dashboard link: {client.dashboard_link}")

Dask dashboard link: http://127.0.0.1:8787/status
